In [1]:
import torch
from torchtext.datasets import AG_NEWS

In [13]:
train_iter = iter(AG_NEWS(split='train'))

# def tokenize(label, line):
#     return line.split()

# tokens = []
# for label, line in train_iter:
#     tokens += tokenize(label, line)

In [32]:
next(train_iter)

(3,
 'US trade deficit swells in June The US trade deficit has exploded 19 to a record \\$55.8bn as oil costs drove imports higher, according to a latest figures.')

In [33]:
# 여기서부턴 https://teddylee777.github.io/pytorch/torchtext-tutorial/
from torchtext.data.utils import get_tokenizer

In [34]:
tokenizer = get_tokenizer('basic_english', language='en')

In [36]:
tokenizer("I'd like to learn torchtext")

['i', "'", 'd', 'like', 'to', 'learn', 'torchtext']

In [39]:
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /Users/jaeha/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [40]:
word_tokenize("I'd like to learn torchtext")

['I', "'d", 'like', 'to', 'learn', 'torchtext']

In [45]:
# 여기서부턴 https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [46]:
tokenizer = get_tokenizer('basic_english')
train_iter = AG_NEWS(split='train')

In [49]:
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)  # yield는 return과 달리 generator를 반환함
        
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [55]:
vocab(['here', 'is', 'an', 'example', 'unknownword', 'unknownwordd'])

[475, 21, 30, 5297, 0, 0]

In [56]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1

In [57]:
text_pipeline('here is the an example')

[475, 21, 2, 30, 5297]

In [58]:
label_pipeline('10')

9

---
# 다시 시작해보자

In [1]:
# 레퍼런스: https://pytorch.org/text/0.12.0/datasets.html#torchtext-datasets
# import datasets

import torch
from torchtext.datasets import AG_NEWS, IMDB

agnews_train = AG_NEWS(split='train')
imdb_train = IMDB(split='train')

for idx, (label, line) in enumerate(agnews_train):
    # train: 120000개(0~119999), test: 7600개(0~7599)
    if idx == 120000-1:
        print(label, '|', line, '\n')
        break
        

def space_tokenize(label, line):
    return line.split()

agnews_tokens = []
for label, line in agnews_train:
    agnews_tokens += space_tokenize(label, line) # 아래엔 이거 대신 torchtext tokenizer 써보겠음
    
print(agnews_tokens[0:10], len(agnews_tokens))

2 | Nets get Carter from Raptors INDIANAPOLIS -- All-Star Vince Carter was traded by the Toronto Raptors to the New Jersey Nets for Alonzo Mourning, Eric Williams, Aaron Williams, and a pair of first-round draft picks yesterday. 



/Users/jaeha/opt/anaconda3/lib/python3.9/site-packages/torch/utils/data/datapipes/iter/combining.py:248: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


['Wall', 'St.', 'Bears', 'Claw', 'Back', 'Into', 'the', 'Black', '(Reuters)', 'Reuters'] 4541694


In [2]:
# 이번엔 torchtext tokenizer. 딱봐도 뭐가 다른지 알겠지?

from torchtext.data.utils import get_tokenizer

torch_tokenizer = get_tokenizer('basic_english', language='en')
def torch_tokenize(label, line):
    return torch_tokenizer(line)

agnews_tokens = []
for label, line in agnews_train:
    agnews_tokens += torch_tokenize(label, line)

print(agnews_tokens[0:10], len(agnews_tokens))

['wall', 'st', '.', 'bears', 'claw', 'back', 'into', 'the', 'black', '('] 5193609


In [3]:
# 이번엔 nltk tokenizer. 위에 연습해놓은 튜토리얼이랑 똑같음

import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize

def nltk_tokenize(label, line):
    return word_tokenize(line)

agnews_tokens = []
for label, line in agnews_train:
    agnews_tokens += nltk_tokenize(label, line)
    
print(agnews_tokens[0:10], len(agnews_tokens))

[nltk_data] Downloading package punkt to /Users/jaeha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['Wall', 'St.', 'Bears', 'Claw', 'Back', 'Into', 'the', 'Black', '(', 'Reuters'] 5273378


# TF-IDF Vectorize

In [4]:
# 이제 드디어 TfidfVectorizer를 사용해보자. AG_NEWS의 데이터만 쭉 전달하면 됨.
# tokenizer는 TfidfVectorizer 함수 인자로 callable tokenizer(위에 만든 tokenize함수들) 넣어주면됨

# 레퍼런스 : https://scikit-learn.org/stable/auto_examples/text/plot_document_clustering.html
# 참고로 이전에 이거 할때 확인해봤는데 20newsgroups(sklearn내장데이터셋) dataset.data 항목은 그냥 str list였음. 고로 똑같음.


In [5]:
# 먼저 해볼 것은 짱간단한 https://wikidocs.net/31698 예제
# 대신 corpus에는 AG_NEWS의 데이터를 다 넣는거지! 짱오래걸릴듯! 돌려놓고 담배피면 개쌉꿀

from sklearn.feature_extraction.text import TfidfVectorizer

corpus = []
for label, line in agnews_train:
    corpus.append(line)

print(corpus[0:5])
tfidfv = TfidfVectorizer().fit(corpus)

print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

["Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.', "Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\\about the economy and the outlook for earnings are expected to\\hang over the stock market next week during the depth of the\\summer doldrums.", 'Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\\flows from the main pipeline in southern Iraq after\\intelligence showed a rebel militia could strike\\infrastructure, an oil official said on Saturday.', 'Oil prices soar to all-time record, posing new menace to US economy (AFP) 

In [6]:
# ㅋ 뭐냐 1초만에 되네

tfidf_array = tfidfv.transform(corpus).toarray()
print(tfidfv)
print(tfidf_array[4])

TfidfVectorizer()
[0. 0. 0. ... 0. 0. 0.]


In [13]:
print(type(tfidf_array[8]), len(tfidf_array[8]))
words = []
for idx, val in enumerate(tfidf_array[8]):
    if val!=0:
        print(idx, val)
        word = [key for key, value in tfidfv.vocabulary_.items() if value == idx]
        print(idx, '->', word)
        words += word

<class 'numpy.ndarray'> 65006
1 0.08429081699404549
1 -> ['000']
1138 0.12385275653006199
1138 -> ['32']
1254 0.09801845222350558
1254 -> ['36']
2007 0.12075300221023062
2007 -> ['70']
2744 0.06925973284675019
2744 -> ['about']
3503 0.11075898404399574
3503 -> ['after']
4508 0.052442725242887366
4508 -> ['an']
4805 0.10762677867725062
4805 -> ['annual']
5594 0.04750401759634723
5594 -> ['as']
5905 0.14263349697924355
5905 -> ['at']
6270 0.10855542643298725
6270 -> ['available']
7111 0.11597235110795233
7111 -> ['base']
7298 0.20778527863649304
7298 -> ['bazil']
7750 0.11841998087810832
7750 -> ['benefits']
9748 0.20778527863649304
9748 -> ['brochures']
10300 0.06263409535598183
10300 -> ['but']
10337 0.11322996322986979
10337 -> ['buying']
10358 0.04942367919712267
10358 -> ['by']
13219 0.1550248118108446
13219 -> ['com']
13334 0.11319324551001227
13334 -> ['commercial']
15750 0.13954704396409948
15750 -> ['danny']
16866 0.15105004332812597
16866 -> ['desk']
18580 0.10213535284503267
1

In [14]:
words2 = []
for idx, (label, line) in enumerate(agnews_train):
    if idx == 8:
        print(label, '|', line, '\n')
        words2 = torch_tokenize(label, line)
        break

3 | Safety Net (Forbes.com) Forbes.com - After earning a PH.D. in Sociology, Danny Bazil Riley started to work as the general manager at a commercial real estate firm at an annual base salary of  #36;70,000. Soon after, a financial planner stopped by his desk to drop off brochures about insurance benefits available through his employer. But, at 32, "buying insurance was the furthest thing from my mind," says Riley. 



In [15]:
words.sort()
print(len(words), words)

words2.sort()
print(len(words2), words2)

58 ['000', '32', '36', '70', 'about', 'after', 'an', 'annual', 'as', 'at', 'available', 'base', 'bazil', 'benefits', 'brochures', 'but', 'buying', 'by', 'com', 'commercial', 'danny', 'desk', 'drop', 'earning', 'employer', 'estate', 'financial', 'firm', 'forbes', 'from', 'furthest', 'general', 'his', 'in', 'insurance', 'manager', 'mind', 'my', 'net', 'of', 'off', 'ph', 'planner', 'real', 'riley', 'safety', 'salary', 'says', 'sociology', 'soon', 'started', 'stopped', 'the', 'thing', 'through', 'to', 'was', 'work']
88 ['#36', '(', ')', ',', ',', ',', ',', ',', ',', '-', '.', '.', '.', '.', '.', '.', '.', '000', '32', '70', 'a', 'a', 'a', 'about', 'after', 'after', 'an', 'annual', 'as', 'at', 'at', 'at', 'available', 'base', 'bazil', 'benefits', 'brochures', 'but', 'buying', 'by', 'com', 'com', 'commercial', 'd', 'danny', 'desk', 'drop', 'earning', 'employer', 'estate', 'financial', 'firm', 'forbes', 'forbes', 'from', 'furthest', 'general', 'his', 'his', 'in', 'insurance', 'insurance', 'ma

In [16]:
# 대충 여기있는 샘플크기처럼 출력도 해보자면 : https://scikit-learn.org/stable/auto_examples/text/plot_document_clustering.html#feature-extraction-using-tfidfvectorizer

print(f"n_samples: {tfidf_array.shape[0]}, n_features: {tfidf_array.shape[1]}")

n_samples: 120000, n_features: 65006


이게 해보니까 단어가 너무 많아서(65006개..) 커널이 뻗어버림. Stemming, Lemmatization을 해주자.

불용어(stopwords)도 처리해야됨. regular expression(정규표현식)으로 특수문자나 숫자 등 영어 아닌것도 다 제거해야됨

## 텍스트 전처리(Stemming, Lemmatization, Stopword제거, 정규표현식으로 영단어만 찾기 등)

In [39]:
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

# stopword 다운해줘야됨
import nltk
nltk.download('stopwords')

stopwords_list = stopwords.words('english')
#print('stopwords: ', stopwords_list)

import re
# 영어 소문자만 추려볼게?
myre = re.compile("[a-z]")
print('651job', bool(myre.search('651job')))

# 토크나이저에 다 넣어부려
def tokenize(line):
    tokens = torch_tokenize(0, line.replace('\\', ' '))
    stems = []
    for word in tokens:
        word = PorterStemmer().stem(word)
        if word not in stopwords_list:    # stopword detection
            if bool(myre.search(word)):    # 'english word' detection
                stems.append(word)
            else:
                pass#print('detected non-words: , word)
        else:
            pass#print('detected stopword: ', word)
    return stems

651job True


[nltk_data] Downloading package stopwords to /Users/jaeha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [40]:
print(tokenize)

for idx, (label, line) in enumerate(agnews_train):
    if idx == 8:
        stems = tokenize(line)
        stems.sort()
        print(len(stems), stems)
        print('중복제거개수 : ', len(set(stems)))
        break

<function tokenize at 0x2955deb80>
44 ['annual', 'avail', 'base', 'bazil', 'benefit', 'brochur', 'buy', 'com', 'com', 'commerci', 'danni', 'desk', 'drop', 'earn', 'employ', 'estat', 'financi', 'firm', 'forb', 'forb', 'furthest', 'gener', 'hi', 'hi', 'insur', 'insur', 'manag', 'mind', 'net', 'ph', 'planner', 'real', 'riley', 'riley', 'safeti', 'salari', 'say', 'sociolog', 'soon', 'start', 'stop', 'thing', 'wa', 'work']
중복제거개수 :  39


43->31로 줄임 (중복제거29->24)

## 본격 돌린다 TF-IDF

In [41]:
# tokenizer 추가하고 불용어(stopwords) 버려서 TfidfVectorizer 다시 돌리기!

# from sklearn.feature_extraction.text import TfidfVectorizer

corpus = []    # corpus는 똑같지뭐
for label, line in agnews_train:
    corpus.append(line)

print(corpus[0:5])
tfidfv2 = TfidfVectorizer(    # 이번엔 tokenizer, max_df, min_df, stop_words 다 추가해보자
    tokenizer=tokenize, 
    max_df=0.5,
    min_df=5,
    stop_words="english",
).fit(corpus)

tfidf_array2 = tfidfv2.transform(corpus).toarray()
print(tfidfv2.vocabulary_)

["Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.', "Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\\about the economy and the outlook for earnings are expected to\\hang over the stock market next week during the depth of the\\summer doldrums.", 'Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\\flows from the main pipeline in southern Iraq after\\intelligence showed a rebel militia could strike\\infrastructure, an oil official said on Saturday.', 'Oil prices soar to all-time record, posing new menace to US economy (AFP) 

In [42]:
print(tfidf_array2.shape[0], tfidf_array2.shape[1])
print('총 단어 수: ', len(tfidfv2.vocabulary_))
# max_df, min_df 안넣었을 땐 61998개였음

120000 19362
총 단어 수:  19362


In [43]:
# 이건 좀 걸리네 ㅋ 아까했던거 다시 다 해보자면

print(type(tfidf_array2[8]), len(tfidf_array2[8]))
words3 = []
for idx, val in enumerate(tfidf_array2[8]):
    if val!=0:
        print(idx, val)
        word = [key for key, value in tfidfv2.vocabulary_.items() if value == idx]
        print(idx, '->', word)
        words3 += word
        
#

words.sort()
print('words: ', len(words), words)

words2.sort()
print('words2: ', len(words2), words2)

words3.sort()
print('words3: ', len(words3), words3)

# 대충 여기있는 샘플크기처럼 출력도 해보자면 : https://scikit-learn.org/stable/auto_examples/text/plot_document_clustering.html#feature-extraction-using-tfidfvectorizer

print(f"n_samples of tfidf_array2: {tfidf_array2.shape[0]}, n_features of tfidf_array2: {tfidf_array2.shape[1]}")

<class 'numpy.ndarray'> 19362
1190 0.1343285102786727
1190 -> ['annual']
1655 0.1336890289174082
1655 -> ['avail']
1905 0.12646806928378726
1905 -> ['base']
2085 0.13744459833213613
2085 -> ['benefit']
2885 0.10830978668705533
2885 -> ['buy']
3759 0.22399074131278185
3759 -> ['com']
3791 0.14057969369101755
3791 -> ['commerci']
4469 0.1765923817070878
4469 -> ['danni']
4773 0.18994279871776482
4773 -> ['desk']
5230 0.11600980279488961
5230 -> ['drop']
5375 0.11005474988565177
5375 -> ['earn']
5621 0.14708880587086964
5621 -> ['employ']
5813 0.16850346014841344
5813 -> ['estat']
6323 0.11911195953597727
6323 -> ['financi']
6359 0.10911596467802726
6359 -> ['firm']
6569 0.3410069988501424
6569 -> ['forb']
7042 0.10857523010429702
7042 -> ['gener']
7903 0.14390159082077125
7903 -> ['hi']
8663 0.2622305474938267
8663 -> ['insur']
10387 0.10465571616967953
10387 -> ['manag']
10998 0.15483980907922132
10998 -> ['mind']
11630 0.1192667891134827
11630 -> ['net']
12899 0.2325743704254918
12899 

**결론 : 120000개 샘플 텍스트에서 19362개의 핵심 단어를 추출하고, 문서별로 이 단어의 빈도를 수치화하여 vector로 표기함.**

In [55]:
# sklearn 튜토리얼에 나온..것과 비슷하게 tfidf_array2의 non-zero값 비율의 수치화
import numpy as np

print("전체 elements중 non-zero elements 비율: ", f"{np.count_nonzero(tfidf_array2) / np.prod(tfidf_array2.shape):.3f}")
print(np.count_nonzero(tfidf_array2), np.prod(tfidf_array2.shape), 120000*19362)

전체 elements중 non-zero elements 비율:  0.001
2401948 2323440000 2323440000


# 이제 KMeans로 Clustering

In [57]:
print(min([min(x) for x in tfidf_array2]), max([max(x) for x in tfidf_array2]))

0.0 0.9177081305893594


In [ ]:
# AGNEWS_K = 4
# # 일단 AGNEWS는 4개 카테고리니까 4개로 설정해서 보자

# from sklearn.cluster import KMeans

# # 클러스터 4개 다른건 다 디폴트로 해버리자
# kmeans = KMeans(n_clusters = AGNEWS_K)
# kmeans.fit(tfidf_array)


In [58]:
# 일단 다시 튜토리얼로 돌아가서 해봐 https://scikit-learn.org/stable/auto_examples/text/plot_document_clustering.html#clustering-sparse-data-with-k-means

from sklearn.cluster import KMeans

for seed in range(5):
    kmeans = KMeans(
        n_clusters=4,  # AGNEWS가 4개니깐!
        max_iter=100,
        n_init=1,
        random_state=seed,
    ).fit(tfidf_array2)
    cluster_idf, cluster_sizes = np.unique(kmeans.labels_, return_counts=True)
    print(f"# of elements asigned to each cluster: {cluster_sizes}")

# of elements asigned to each cluster: [37094  9175  6098 67633]
# of elements asigned to each cluster: [ 9345  4999 36733 68923]
# of elements asigned to each cluster: [80020 19894  6473 13613]
# of elements asigned to each cluster: [ 7590  3211 76212 32987]
# of elements asigned to each cluster: [ 9493 34526  6250 69731]



NameError: name 'category_sizes' is not defined

In [62]:
agnews_labels = []
for label, line in agnews_train:
    agnews_labels.append(label)
    
print()
print(
    "True number of documents in each category according to the class labels: \n"
    f"{[(x,agnews_labels.count(x)) for x in range(1,5)]}"
)


True number of documents in each category according to the class labels: 
[(1, 30000), (2, 30000), (3, 30000), (4, 30000)]


In [ ]:
# n_init=5로 해서 top terms를 한번 출력해보자.

---

In [63]:
# 튜토리얼의 fit and evaluate 함수 그대로 해보자(시간 측정, 각종 score 출력 등 포함됨)

from collections import defaultdict
from sklearn import metrics
from time import time

evaluations = []
evaluations_std = []


def fit_and_evaluate(km, X, name=None, n_runs=5):
    name = km.__class__.__name__ if name is None else name

    train_times = []
    scores = defaultdict(list)
    for seed in range(n_runs):
        km.set_params(random_state=seed)
        t0 = time()
        km.fit(X)
        train_times.append(time() - t0)
        scores["Homogeneity"].append(metrics.homogeneity_score(labels, km.labels_))
        scores["Completeness"].append(metrics.completeness_score(labels, km.labels_))
        scores["V-measure"].append(metrics.v_measure_score(labels, km.labels_))
        scores["Adjusted Rand-Index"].append(
            metrics.adjusted_rand_score(labels, km.labels_)
        )
        scores["Silhouette Coefficient"].append(
            metrics.silhouette_score(X, km.labels_, sample_size=2000)
        )
    train_times = np.asarray(train_times)

    print(f"clustering done in {train_times.mean():.2f} ± {train_times.std():.2f} s ")
    evaluation = {
        "estimator": name,
        "train_time": train_times.mean(),
    }
    evaluation_std = {
        "estimator": name,
        "train_time": train_times.std(),
    }
    for score_name, score_values in scores.items():
        mean_score, std_score = np.mean(score_values), np.std(score_values)
        print(f"{score_name}: {mean_score:.3f} ± {std_score:.3f}")
        evaluation[score_name] = mean_score
        evaluation_std[score_name] = std_score
    evaluations.append(evaluation)
    evaluations_std.append(evaluation_std)

In [ ]:
# 튜토리얼대로 n_init=5로 해서 제대로 fit and evaluate 해보자

kmeans = KMeans(
    n_clusters=4,    # AGNEWS가 4개니깐!
    max_iter=100,
    n_init=5,
)

fit_and_evaluate(kmeans, tfidf_array2, name="KMeans\non tf-idf vectors")

In [ ]:
print(kmeans.cluster_centers_)

## LSA 차원축소

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

lsa = make_pipeline(TruncatedSVD(n_components=100), Normalizer(copy=False))
t0 = time()
lsa_array2 = lsa.fit_transform(tfidf_array2)
explained_variance = lsa[0].explained_variance_ratio_.sum()

print(f"LSA done in {time() - t0:.3f} s")
print(f"Explained variance of the SVD step: {explained_variance * 100:.1f}%")

In [ ]:
kmeans_lsa = KMeans(
    n_clusters=4,
    max_iter=100,
    n_init=1,
)

fit_and_evaluate(kmeans_lsa, lsa_array2, name="KMeans\nwith LSA on tf-idf vectors")

In [ ]:
from sklearn.cluster import MiniBatchKMeans

minibatch_kmeans = MiniBatchKMeans(
    n_clusters=4,
    n_init=1,
    init_size=1000,
    batch_size=1000,
)

fit_and_evaluate(
    minibatch_kmeans,
    lsa_array2,
    name="MiniBatchKMeans\nwith LSA on tf-idf vectors",
)

## 클러스터별 Top Terms

In [ ]:
original_space_centroids = lsa[0].inverse_transform(kmeans.cluster_centers_)
order_centroids = original_space_centroids.argsort()[:, ::-1]
terms = tfidfv2.get_feature_names_out()

for i in range(4):
    print(f"Cluster {i}: ", end="")
    for ind in order_centroids[i, :10]:
        print(f"{terms[ind]} ", end="")
    print()